In [73]:
import numpy as np
import numpy.linalg as LA
from scipy.linalg import block_diag

Size Guide:

* $n$ - number of training examples
* $d$ - feature dimension
* $r$ - rank of $X$
* $P$ - total enumerations of hidden neurons / ReLU activations
* $P_S$ - number of hidden neurons / samples of $D_i$ matrices

Matrices
* $X \in \mathbb{R}^{n \times d}$ - training data matrix
* $D_i \in \mathbb{R}^{n \times n}, \ i = 1,\dots,P_S$ - diagonal matrices sampling hidden layer activations
* $F_i = D_i X \in \mathbb{R}^{n \times d}, \ i = 1,\dots,P_S$
* $F = \begin{bmatrix} F_1 & \dots & F_{P_S} & -F_1 & \dots & -F_{P_S} \end{bmatrix} \in \mathbb{R}^{n \times (2 d P_S)}, \ i = 1,\dots,P_S$

In [82]:
### user inputs 
n,d = 100,5

# X - data matrix in n x d
X = np.random.randn(n,d)
# y are targets
y = np.random.randn(n,1)

r = np.linalg.matrix_rank(X)

# number of hidden neurons in convex network
P_S = 10

# parameters (TODO: parameterize)
rho = 1e-5
step = 1e-5
beta = 1e-5

In [36]:
### TODO: figure out better way to sample D matrices

h = np.random.randn(d, P_S)

# TODO: assert that no duplicate columns
# n x P_S matrix, where each column is the diagonal entries for one D matrix
d_diags = X @ h >= 0

In [83]:
# gather F and G

# # F is P_S x n x d. F[i] = F_i in (n x d)
# F = np.swapaxes(
#         np.swapaxes(
#             np.dstack([np.diag(d_diags[:,i].astype('uint8')) @ X for i in range(P_S)]),
#             2, 0),
#     1, 2)

# F here is n x (2d*P_S) like paper
F = np.hstack([np.hstack([np.diag(d_diags[:,i].astype('uint8')) @ X for i in range(P_S)]),
    np.hstack([-np.diag(d_diags[:,i].astype('uint8')) @ X for i in range(P_S)[::-1]])])


# G is block diagonal 2*n*P_S x 2*d*P_s
G = block_diag(*[(2 * np.diag(d_diags[:,i].astype('uint8')) - np.eye(n)) @ X for i in range(P_S)] * 2)

# dual variables
lam = np.zeros((2*d*P_S,1))
nu = np.zeros((2*d*P_S,1))

# weights

# u contains u1 ... uP, z1... zP in one long vector
u = np.zeros((d * P_S * 2))
# v contrains v1 ... vP, w1 ... wP in one long vector
v = np.zeros((d * P_S * 2))

# slacks
s = np.zeros((P_S * n))
t = np.zeros((P_S * n))
for i in range(P_S):
    G_i = (2 * np.diag(d_diags[:,i].astype('uint8')) - np.eye(n)) @ X
    # s_i = G_i v_i
    s[i] = G_i @ v[i*d:(i+1)*d]
    # t_i = G_i w_i
    t[i] = G_i @ v[(i+P_S)*d:(i+P_S+1)*d]

# precomputes: A in 2dP_s x 2dP_s
A = np.eye(2*d*P_S) + F.T @ F / rho + G.T @ G
# cholesky factorization
L = LA.cholesky(A)

b_1 = F.T @ y / rho

(100, 100)

In [53]:

# updates 

# update u

# updates of v
for i in range(P_S):
    lam_1i = lam[d*i:d*(i+1)]
    v[i] = np.maximum(1 - beta/(rho * LA.norm(u[i] + lam_1i)), 0)
    v[i] *= (u[i] + lam_1i)

# updates of w
for i in range(P_S):
    lam_2i = lam[d*(i+P_S):d*(i+P_S+1)]
    w[i] = np.maximum(1 - beta/(rho * LA.norm(z[i] + lam_2i)), 0)
    w[i] *= (u[i] + lam_2i)

# updates on s
for i in range(P_S):
    s[i] = np.maximum(G[i] @ u[i] + nu[d*i:d*(i+1)], 0)
    t[i] = np.maximum(G[i] @ z[i] + nu[d*(i+P_S):d*(i+P_S+1)], 0)

# dual updates last
lam += lam + step / rho * (u - v)
nu += lam + step / rho * (G @ u - s)

9
8
7
6
5
4
3
2
1
0
